In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label, HoverTool

# Data

In [2]:
#read EBA data regarding capital
df = pd.read_csv('https://www.eba.europa.eu/documents/10180/2027702/tr_oth.csv', encoding='latin1',\
                 usecols=['LEI_Code','Period','Label','Amount'])

In [3]:
#read EBA metadata
metadata_banks= pd.read_excel('https://www.eba.europa.eu/documents/10180/2027702/TR_Metadata.xlsx',\
                              sheetname='Bank list', skiprows=1, usecols=['LEI_Code','Name','Desc_country'])

In [4]:
#prepare CET1 ratio, CET1, Profit/Loss
df_CET1_ratio_fl=df[(df['Label']=='COMMON EQUITY TIER 1 CAPITAL RATIO (transitional period)') & \
                    (df['Period']==201612)][['LEI_Code','Amount']]
df_CET1_ratio_fl.columns=['LEI_Code', 'CET1_ratio_transitional_period']


df_CET1=df[(df['Label']=='COMMON EQUITY TIER 1 CAPITAL (net of deductions and after applying transitional adjustments)') & \
                    (df['Period']==201612)][['LEI_Code','Amount']]
df_CET1.columns=['LEI_Code', 'CET1']


df_profit_loss=df[(df['Label']=='PROFIT OR (-) LOSS BEFORE TAX FROM CONTINUING OPERATIONS') & \
                    (df['Period']==201612)][['LEI_Code','Amount']]
df_profit_loss.columns=['LEI_Code', 'profit_loss']

In [5]:
#merge individual df
df=df_CET1_ratio_fl.merge(df_CET1, left_on='LEI_Code', right_on='LEI_Code', how='outer').merge\
(df_profit_loss, left_on='LEI_Code', right_on='LEI_Code', how='outer').merge\
(metadata_banks, left_on='LEI_Code', right_on='LEI_Code', how='inner')

In [6]:
#remove NA
df=df.dropna()

In [7]:
#calculate ROE
df['ROE'] = df['profit_loss']/df['CET1']

# Chart

In [8]:
source=ColumnDataSource (data=dict(height=df['CET1_ratio_transitional_period'],
                                   weight=df['ROE'],
                                   names=df['Name']
                                  ))
hover=HoverTool(tooltips=
                    [("ROE","@weight"),
                    ("CET1 Ratio","@height"),
                    ("Bank", "@names")]
               )

TITLE="Return on Common Equity and CET1 Ratio as of Dec 2016 (source: EBA)"

p=figure(plot_width=500, plot_height=250, responsive=True, tools=['pan','wheel_zoom','box_zoom', 'reset', hover], title=TITLE,
        logo=None)

p.scatter(x='weight', y='height',  source=source, alpha=0.2)

p.xaxis.axis_label="Return on Common Equity (CET1)"
p.yaxis.axis_label="CET1 Capital Ratio"

labels=LabelSet(x='weight', y='height', text='names', level='glyph', x_offset=5, y_offset=0, text_font_size="6pt", source=source)
label_opts=dict(x=0, y=1, x_units='screen', y_units='screen')

#msg1="---"
#caption1=Label(text=msg1, **label_opts)

p.add_layout(labels)
#p.add_layout(caption1, 'below')

output_notebook()
show(p)

Loading BokehJS ...